# Import the necessary libraries

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import numpy as np

# Define Constants

In [ ]:
# --- Configuration ---
PREPROCESSED_DATA_ROOT = 'Preprocessed_data'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5 # Heart, Oblong, Oval, Round, Square
EPOCHS_PHASE1 = 10 # Initial epochs for training only the new layers
EPOCHS_PHASE2 = 10 # Additional epochs for fine-tuning some base layers
LEARNING_RATE_PHASE1 = 0.001
LEARNING_RATE_PHASE2 = 0.0001 # Lower learning rate for fine-tuning
MODEL_SAVE_PATH = 'models/face_shape_classifier_5_classes.h5'

CLASS_NAMES = sorted(os.listdir(os.path.join(PREPROCESSED_DATA_ROOT, 'training_set')))
print(f"Detected Classes: {CLASS_NAMES}")
print(f"Number of Classes: {len(CLASS_NAMES)}")

Detected Classes: ['Heart', 'Oblong', 'Oval', 'Round', 'Square']
Number of Classes: 5


# Load the preprocessed dataset

In [7]:
print("Loading datasets...")

train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values to 0-1
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No augmentation for validation data, only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    os.path.join(PREPROCESSED_DATA_ROOT, 'training_set'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    classes=CLASS_NAMES
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(PREPROCESSED_DATA_ROOT, 'testing_set'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=CLASS_NAMES
)

print("Datasets loaded successfully.")

Loading datasets...
Found 4009 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Datasets loaded successfully.


# Build the Model (Transfer Learning)

In [8]:
print("Building model...")

# Load MobileNetV2 pre-trained on ImageNet, excluding the top (classification) layer
base_model = MobileNetV2(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False, # Exclude the classification head
    weights='imagenet' # Use pre-trained ImageNet weights
)

# Freeze the base model layers
base_model.trainable = False

# Add custom classification head on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

print("Model Built Successfully..")

Building model...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model Built Successfully..


# Compile the model

In [9]:
print("Compiling model (Phase 1: Training custom layers)...")
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_PHASE1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Callbacks ---
# Save the best model based on validation accuracy
checkpoint_callback_phase1 = ModelCheckpoint(
    MODEL_SAVE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
# Reduce learning rate when validation accuracy stops improving
reduce_lr_phase1 = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2, # Reduce by a factor of 0.2 (i.e., new_lr = old_lr * 0.2)
    patience=3, # If no improvement for 3 epochs, reduce LR
    min_lr=0.00001,
    verbose=1
)
# Early stopping to prevent overfitting
early_stopping_phase1 = EarlyStopping(
    monitor='val_accuracy',
    patience=5, # Stop if no improvement for 5 epochs
    mode='max',
    verbose=1,
    restore_best_weights=True # Restore weights from the best epoch
)

print('Compilation Complete..')

Compiling model (Phase 1: Training custom layers)...
Compilation Complete..


# Model Training

In [10]:
print("Training Phase 1: Training custom layers...")
history_phase1 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback_phase1, reduce_lr_phase1, early_stopping_phase1]
)
print("Phase 1 training complete.")

Training Phase 1: Training custom layers...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.2825 - loss: 1.7754
Epoch 1: val_accuracy improved from -inf to 0.39500, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 1380s 11s/step - accuracy: 0.2827 - loss: 1.7741 - val_accuracy: 0.3950 - val_loss: 1.4300 - learning_rate: 0.0010
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step - accuracy: 0.3818 - loss: 1.4319
Epoch 2: val_accuracy improved from 0.39500 to 0.43700, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 63s 500ms/step - accuracy: 0.3820 - loss: 1.4317 - val_accuracy: 0.4370 - val_loss: 1.3476 - learning_rate: 0.0010
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.4492 - loss: 1.3384
Epoch 3: val_accuracy did not improve from 0.43700
126/126 ━━━━━━━━━━━━━━━━━━━━ 64s 503ms/step - accuracy: 0.4491 - loss: 1.3386 - val_accuracy: 0.4290 - val_loss: 1.3404 - learning_rate: 0.0010
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.4606 - loss: 1.3249
Epoch 4: val_accuracy did not improve from 0.43700
126/126 ━━━━━━━━━━━━━━━━━━━━ 65s 515ms/step - accuracy: 0.4606 - loss: 1.3248 - val_accuracy: 0.4300 - val_loss: 1.3418 - learning_rate: 0.0010
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - accuracy: 0.4620 - loss: 1.3069
Epoch 5: val_accuracy improved from 0.43700 to 0.46100, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 65s 515ms/step - accuracy: 0.4620 - loss: 1.3068 - val_accuracy: 0.4610 - val_loss: 1.2922 - learning_rate: 0.0010
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - accuracy: 0.4642 - loss: 1.2746
Epoch 6: val_accuracy improved from 0.46100 to 0.46400, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 66s 525ms/step - accuracy: 0.4642 - loss: 1.2748 - val_accuracy: 0.4640 - val_loss: 1.2706 - learning_rate: 0.0010
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - accuracy: 0.4858 - loss: 1.2462
Epoch 7: val_accuracy did not improve from 0.46400
126/126 ━━━━━━━━━━━━━━━━━━━━ 79s 503ms/step - accuracy: 0.4858 - loss: 1.2462 - val_accuracy: 0.4630 - val_loss: 1.2940 - learning_rate: 0.0010
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.4900 - loss: 1.2692
Epoch 8: val_accuracy improved from 0.46400 to 0.48300, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 63s 500ms/step - accuracy: 0.4900 - loss: 1.2690 - val_accuracy: 0.4830 - val_loss: 1.2317 - learning_rate: 0.0010
Epoch 9/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step - accuracy: 0.5007 - loss: 1.2149
Epoch 9: val_accuracy did not improve from 0.48300
126/126 ━━━━━━━━━━━━━━━━━━━━ 64s 505ms/step - accuracy: 0.5006 - loss: 1.2151 - val_accuracy: 0.4590 - val_loss: 1.2653 - learning_rate: 0.0010
Epoch 10/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - accuracy: 0.5121 - loss: 1.1973
Epoch 10: val_accuracy did not improve from 0.48300
126/126 ━━━━━━━━━━━━━━━━━━━━ 64s 511ms/step - accuracy: 0.5121 - loss: 1.1974 - val_accuracy: 0.4790 - val_loss: 1.2525 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 8.
Phase 1 training complete.


# Fine Tuning and Retraining

In [11]:
print("\nStarting Phase 2: Fine-tuning the model...")

# Unfreeze some layers of the base model
base_model.trainable = True

# Freeze all layers except the last few (e.g., last 20 layers of MobileNetV2)
for layer in base_model.layers[:-20]: # Unfreeze the last 20 layers, keep earlier ones frozen
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_PHASE2),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Callbacks for Phase 2 ---
checkpoint_callback_phase2 = ModelCheckpoint(
    MODEL_SAVE_PATH, # Overwrite the previous best model if this one is better
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
reduce_lr_phase2 = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=3,
    min_lr=0.000001, # Even lower minimum LR
    verbose=1
)
early_stopping_phase2 = EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    mode='max',
    verbose=1,
    restore_best_weights=True
)

# Continue training from the state after Phase 1
print("Training Phase 2: Fine-tuning...")
history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1 + EPOCHS_PHASE2, # Total epochs will be sum, but it continues from EPOCHS_PHASE1
    initial_epoch=history_phase1.epoch[-1] + 1 if history_phase1.epoch else 0, # Start from where Phase 1 left off
    validation_data=validation_generator,
    callbacks=[checkpoint_callback_phase2, reduce_lr_phase2, early_stopping_phase2]
)

print("\nModel training complete!")
print(f"Final model saved to: {MODEL_SAVE_PATH}")


Starting Phase 2: Fine-tuning the model...
Training Phase 2: Fine-tuning...
Epoch 11/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 516ms/step - accuracy: 0.4224 - loss: 1.4216
Epoch 11: val_accuracy improved from -inf to 0.50200, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 94s 617ms/step - accuracy: 0.4227 - loss: 1.4207 - val_accuracy: 0.5020 - val_loss: 1.3098 - learning_rate: 1.0000e-04
Epoch 12/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 502ms/step - accuracy: 0.5799 - loss: 1.0739
Epoch 12: val_accuracy did not improve from 0.50200
126/126 ━━━━━━━━━━━━━━━━━━━━ 67s 531ms/step - accuracy: 0.5799 - loss: 1.0738 - val_accuracy: 0.4630 - val_loss: 2.0633 - learning_rate: 1.0000e-04
Epoch 13/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step - accuracy: 0.6174 - loss: 0.9689
Epoch 13: val_accuracy improved from 0.50200 to 0.51100, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 66s 523ms/step - accuracy: 0.6175 - loss: 0.9687 - val_accuracy: 0.5110 - val_loss: 1.5809 - learning_rate: 1.0000e-04
Epoch 14/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - accuracy: 0.6827 - loss: 0.8391
Epoch 14: val_accuracy improved from 0.51100 to 0.52300, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 64s 510ms/step - accuracy: 0.6827 - loss: 0.8392 - val_accuracy: 0.5230 - val_loss: 1.5842 - learning_rate: 1.0000e-04
Epoch 15/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.7279 - loss: 0.7395
Epoch 15: val_accuracy improved from 0.52300 to 0.59800, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 63s 503ms/step - accuracy: 0.7279 - loss: 0.7396 - val_accuracy: 0.5980 - val_loss: 1.1276 - learning_rate: 1.0000e-04
Epoch 16/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.7560 - loss: 0.6470
Epoch 16: val_accuracy did not improve from 0.59800
126/126 ━━━━━━━━━━━━━━━━━━━━ 82s 507ms/step - accuracy: 0.7559 - loss: 0.6471 - val_accuracy: 0.5400 - val_loss: 1.5479 - learning_rate: 1.0000e-04
Epoch 17/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - accuracy: 0.7767 - loss: 0.6365
Epoch 17: val_accuracy did not improve from 0.59800
126/126 ━━━━━━━━━━━━━━━━━━━━ 63s 502ms/step - accuracy: 0.7767 - loss: 0.6365 - val_accuracy: 0.5870 - val_loss: 1.3657 - learning_rate: 1.0000e-04
Epoch 18/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.8053 - loss: 0.5315
Epoch 18: val_accuracy improved from 0.59800 to 0.60300, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 64s 504ms/step - accuracy: 0.8053 - loss: 0.5316 - val_accuracy: 0.6030 - val_loss: 1.4437 - learning_rate: 1.0000e-04
Epoch 19/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.8082 - loss: 0.5139
Epoch 19: val_accuracy improved from 0.60300 to 0.61500, saving model to /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 67s 529ms/step - accuracy: 0.8082 - loss: 0.5139 - val_accuracy: 0.6150 - val_loss: 1.3057 - learning_rate: 1.0000e-04
Epoch 20/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step - accuracy: 0.8389 - loss: 0.4539
Epoch 20: val_accuracy did not improve from 0.61500
126/126 ━━━━━━━━━━━━━━━━━━━━ 65s 520ms/step - accuracy: 0.8388 - loss: 0.4541 - val_accuracy: 0.6150 - val_loss: 1.3512 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 19.

Model training complete!
Final model saved to: /content/drive/My Drive/Colab Notebooks/FaceShape_Analyzer/models/face_shape_classifier_5_classes.h5


# Evalution

In [12]:
print("\nEvaluating final model on the validation set...")
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Evaluating final model on the validation set...
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.6847 - loss: 0.9722
Validation Loss: 1.3057
Validation Accuracy: 0.6150
